In [ ]:
# chat with multiple pdfs using llama 2+Pinecone+Langchain

In [ ]:
#install required libraries

In [ ]:
!pip install Langchain
!pip install pincone-client
!pip install sentence-transformers
!pip install pdf2image
!pip install pypdf
!pip install xformers
!pip install bitstandbytes accelerates transformers


In [ ]:
# import required all important library

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from transformers import AutoTokenizer,AutoModelForCausalLM
from transformers import pipeline
import os
import sys


In [ ]:
# load the pdfs files

In [ ]:
!mkdirs pdfs

In [ ]:

!gdown 1hPQlXrX8FbaYaLypxTmeVOFNitbBMlEE -O pdfs/yolov7paper.pdf
!gdown 1vILwiv6nS2wI3chxNabMgry3qnV67TxM -O pdfs/rachelgreecv.pdf

In [ ]:
# extract text from the PDF'S


In [ ]:
loader=PyPDFDirectoryLoader("pdfs")

In [ ]:
print(data)

In [ ]:
# split data into extracted data into text chunks

In [ ]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)

In [ ]:
docs=text_splitter.split_document(data)


In [ ]:
print(len(docs))


In [ ]:
docs[3]

In [ ]:
docs[4]

In [ ]:
# download the embeddings from the huggingface

In [ ]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
query_results=embeddings.embed_query("Hellow-world")
Print(len(query_results))


In [ ]:
# intializing the pinecone

In [ ]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY','XXXXXXXXXXXXX')
PINECONE_API_ENV=os.environ.get('PINECONE_API_ENV','XXXXXXXXXXXXX')


In [ ]:
# initialize the pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV,

)

In [ ]:
index_name="langchainpinecone"

In [ ]:
# crate embeddings for the each of text chunks
docssearch=Pinecone.from_texts([t.page_content for t in docs],embeddings,index_name=index_name)

In [ ]:
# if already having the index, can be loaded
# docsearch=Pinecone.from_existing_index(index_name,embeddings)

In [ ]:
# similaririty search

In [ ]:
query="YOLOv7 outperforms which models"


In [ ]:
docs = docssearch.similarity_search(query,k=4)
docs

In [ ]:
# creating a llama2 model model wrapper

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",use_auth_token=True)


In [ ]:
model=AutomodelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                           device_map='auto',
                                           torch_dtype=torch.flaot16,
                                           use_auth_token=True,
                                           load_in_bit=True)


In [ ]:
pipe=pipeline("text-generations",
              model=model,
              tokenizer=tokenizer,
              torch_dtype=torch.bfloat16,
              device_map="auto",
              max_new_tokens=512,
              do_sample=True,
              top_k=30,
              num_return_sequences=1,
              eos_token_id=tokenizer.eos_token_id)


In [ ]:
llm=HuggingFacePipeline(pipeline=pipe,model_kwargs={'temperature':0.1})


In [ ]:
# crete a prompt templates
SYSTEM_PROMPTS = """use the following pieces of context to answer the question at the end.
                    if you dont know the answer,just say that you don't know,dont try to makeup answer.
                 """

In [ ]:
B_INST,E_INST = '[INST]','[/INST]'
B_SYS,E_SYS = '<>\n',"\n<>\n\n"

In [ ]:
SYSTEM_PROMPT = B_SYS + SYSTEM_PROMPT + E_SYS

In [ ]:
instruction="""
              {context}
              Question:{question}
            """


In [ ]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [ ]:
templates

In [ ]:
prompt=PromptTemplate(template=template,input_varible=["context","questions"])

In [ ]:
qa_chain=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuffs",
    retriever=docsearch.as_retriever(search_kwargs={"k":2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt},
)

In [ ]:
result=qa_chain("YOLO7 IS USED FOR?")


In [ ]:
result['result']

In [ ]:
while True:
  user_input=input(f"prompts:")
  if user_input=='exit':
    print("existing")
    sys.exit()
  if user_input=='':
    continue
  result=qa_chain({'query':user_input})
  print(f"Answer:{result['result']}")